# New CD31 model apply for Yuyi Wang
## 03/30/23 

### Author: Andy D. Tran, CCR Microscopy Core, LCBG, CCR, NCI

In [1]:
# Import libraries----------------------------------------------------------------------------------------

import os 
import numpy as np 
import cv2 
import pandas as pd 
import pickle 
import re 
import czifile as zis 
import napari

from scipy import ndimage as nd
from tifffile import imread, imwrite 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier 
from sklearn import metrics 
from datetime import datetime 
from skimage.util import img_as_ubyte 
from skimage.filters.rank import entropy 
from skimage.morphology import disk, binary_opening, binary_dilation
from skimage.measure import regionprops, label 
from skimage.segmentation import expand_labels
from tqdm import tqdm 

In [2]:
# Define functions----------------------------------------------------------------------------------------

def feature_extract(img):
    df = pd.DataFrame() 
    
    print('Add original pixel values')
    org = img.reshape(-1)
    df['org_image'] = org 
    
    print('Add Gabor features')
    num = 1 
    kernels = [] 
    
    for theta in range(2):
        theta = theta / 4 * np.pi 
        for sigma in (1, 3):
            for lamda in np.arange(0, np.pi, np.pi / 4):
                for gamma in (0.05, 0.5):
                        
                    gabor_label = 'Gabor' + str(num)
                    ksize = 3
                    kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, 0, ktype = cv2.CV_32F)
                    kernels.append(kernel)
                        
                    fimg = cv2.filter2D(img, cv2.CV_16UC3, kernel)
                    filtered_img = fimg.reshape(-1)
                        
                    df[gabor_label] = filtered_img
                    num += 1
                        
    print('Add Median filter')
    for sigma in range(2, 8):
        print('Sigma = ' + str(sigma))
        
        median_tmp = nd.median_filter(img, size = sigma)
        median_img = median_tmp.reshape(-1)
        median_header = 'Median_sigma_' + str(sigma)
        
        df[median_header] = median_img 
        
    print('Add Entropy filter')
    for sigma in range(2, 8):
        print('Sigma = ' + str(sigma))
        
        entropy_tmp = entropy(img, disk(sigma))
        entropy_img = entropy_tmp.reshape(-1)
        entropy_header = 'Entropy_sigma_' + str(sigma)
        
        df[entropy_header] = entropy_img
        
    return df 

    print('Feature extraction done!')
    
def roi_quant(mask, img):
    df = pd.DataFrame() 
    
    id = [] 
    int = []
    area = [] 
    total_area = [] 
    
    roi_props = regionprops(mask, intensity_image = img) 
    
    for roi in range(len(roi_props)):
        id.append(roi_props[roi].label)
        int.append(roi_props[roi].mean_intensity)
        area.append(roi_props[roi].area)
        total_area = img.shape[0] * img.shape[1]
        
    df['id'] = id 
    df['int'] = int 
    df['area'] = area 
    df['total_area'] = total_area 
    
    return df 

def model_apply(img, model):
    print('Segmenting')
    df = feature_extract(img)
    tmp = model.predict(df)
    mask = tmp.reshape(img.shape)
    mask = binary_opening(mask)
    mask = binary_dilation(mask)
    mask = np.uint16(label(mask))
    print('Segmentation complete: ', datetime.now())
    
    return mask

def object_apply(img, orig_int, model):
    print('Object classification')
    df = roi_quant(img, orig_int)
    id_array = df['id'].to_numpy() 
    df = df.drop(labels = ['id', 'total_area'], axis = 1)
    print('ML object classification started: ', datetime.now())
    object_tmp = model.predict(df)
    id_array = np.append(id_array, 0)
    object_tmp = np.append(object_tmp, 0)
    object_df = pd.DataFrame() 
    object_df['id'] = id_array 
    object_df['object_class'] = object_tmp 
    
    mask_id = img.reshape(-1)
    mask_df = pd.DataFrame() 
    mask_df['id'] = mask_id 
    mask_df = pd.merge(mask_df, object_df, how = 'left', on = ['id'])
    mask_df['object_mask'] = np.where(mask_df['object_class'] == 1, 1, 0)
    mask = mask_df['object_class'].to_numpy() 
    mask = mask.reshape(img.shape)
    mask = np.uint16(label(mask))
    print('Object classification complete: ', datetime.now())
    
    return mask 

def main(tmp_path):
    tmp_list = os.listdir(tmp_path)
    img_list = [] 
    for tmp_name in tmp_list:
        if re.search('.czi', tmp_name):
            img_list.append(tmp_name)
            
        for img_name in img_list:
            print(img_name)
            img_path = os.path.join(tmp_path, img_name)
            czi_img = zis.CziFile(img_path)
            img = czi_img.asarray() 
            img = img[0, 0, :, 0, 0, :, :, 0]
            print(img.shape)
            
            cd31_img = img[0, :, :]
            
            cd31_mask = model_apply(cd31_img, cd31_model)
            #cd31_mask = object_apply(cd31_mask, cd31_img, cd31_object)
            cd31_mask_path = os.path.join(tmp_path, img_name.replace('.czi', '_cd31_mask.tif'))
            imwrite(cd31_mask_path, cd31_mask)
            
        print('Quantification complete! ', datetime.now())

In [8]:
# Main function-----------------------------------------------------------------------------------------

base_path = '/Volumes/LECIMAGE/Analysis/[NCI] [LCO] Giovanna Tosato/Yuyi Wang/image/LSM880'
cd31_model_path = '/Volumes/LECIMAGE/Analysis/[NCI] [LCO] Giovanna Tosato/Yuyi Wang/training/cd31/cd31_model_02'
cd31_object_path = '/Volumes/LECIMAGE/Analysis/[NCI] [LCO] Giovanna Tosato/Yuyi Wang/training/cd31_object_classifier/cd31_object_model_02'

cd31_model = pickle.load(open(cd31_model_path, 'rb'))
cd31_object = pickle.load(open(cd31_object_path, 'rb'))


In [10]:
set_list = os.listdir(base_path)
print(set_list)

set_list.remove('.DS_Store')
print(set_list)

['01_03', '.DS_Store', 'foxo_test_02', 'foxo_test', 'foxo_test_02_13', '02_13']
['01_03', 'foxo_test_02', 'foxo_test', 'foxo_test_02_13', '02_13']


In [11]:
set_path = os.path.join(base_path, set_list[0])
tmp_list = os.listdir(set_path)
img_list = [] 
for tmp in tmp_list:
    if re.search('.czi', tmp):
        img_list.append(tmp)
print(img_list)
print(len(img_list))

['B16F10-shSHP2-control-2-Ki67-CD31.czi', 'B16F10-shSHP2-GDC1-Ki67-CD31.czi', 'B16F10-shSHP2-GDC5-caspase 3-CD31.czi', 'B16F10-shSHP2-control-2-caspase 3-CD31.czi', 'B16F10-shSHP2-GDC2-Ki67-CD31.czi', 'B16F10-shSHP2-control-5-caspase 3-CD31.czi', 'test.czi', 'B16F10-shSHP2-control-3-Ki67-CD31.czi', 'B16F10-shSHP2-control-1-Ki67-CD31.czi', 'B16F10-shSHP2-GDC1-caspase 3-CD31.czi']
10


In [12]:
img_name = img_list[3]
img_path = os.path.join(set_path, img_name)
czi_img = zis.CziFile(img_path)
img = czi_img.asarray() 
#img = img[0, :, 0, 0, :, :, 0]
img = img[0, 0, :, 0, 0, :, :, 0]
print(img.shape)

(3, 947, 1816)


In [13]:
dapi_img = img[2, :, :]
exp_img = img[1, :, :]
cd31_img = img[0, :, :]

In [14]:
viewer = napari.view_image(cd31_img, name = 'CD31')
viewer.layers['CD31'].contrast_limits = (0, 500)
exp_layer = viewer.add_image(exp_img, name = 'Exp')
viewer.layers['Exp'].contrast_limits = (0, 200)
dapi_layer = viewer.add_image(dapi_img, name = 'DAPI')
viewer.layers['DAPI'].contrast_limits = (0, 500)

In [15]:
viewer.close()

In [16]:
cd31_mask = model_apply(cd31_img, cd31_model)

Segmenting
Add original pixel values
Add Gabor features
Add Median filter
Sigma = 2
Sigma = 3
Sigma = 4
Sigma = 5
Sigma = 6
Sigma = 7
Add Entropy filter
Sigma = 2


/Users/tranad/anaconda3/envs/napari-env/lib/python3.11/site-packages/skimage/filters/rank/generic.py:268: UserWarning: Bad rank filter performance is expected due to a large number of bins (3248), equivalent to an approximate bitdepth of 11.7.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Sigma = 3


/Users/tranad/anaconda3/envs/napari-env/lib/python3.11/site-packages/skimage/filters/rank/generic.py:268: UserWarning: Bad rank filter performance is expected due to a large number of bins (3248), equivalent to an approximate bitdepth of 11.7.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Sigma = 4


...

Sigma = 6


/Users/tranad/anaconda3/envs/napari-env/lib/python3.11/site-packages/skimage/filters/rank/generic.py:268: UserWarning: Bad rank filter performance is expected due to a large number of bins (3248), equivalent to an approximate bitdepth of 11.7.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Sigma = 7


/Users/tranad/anaconda3/envs/napari-env/lib/python3.11/site-packages/skimage/filters/rank/generic.py:268: UserWarning: Bad rank filter performance is expected due to a large number of bins (3248), equivalent to an approximate bitdepth of 11.7.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Segmentation complete:  2024-03-01 13:04:04.716447


In [571]:
#cd31_mask_path = os.path.join(set_path, img_name.replace('.czi', '_cd31_mask.tif'))
#cd31_mask = imread(cd31_mask_path)

In [17]:
cd31_mask = np.uint16(label(cd31_mask))
cd31_mask_tmp = object_apply(cd31_mask, cd31_img, cd31_object)

Object classification
ML object classification started:  2024-03-01 13:04:21.740733
Object classification complete:  2024-03-01 13:04:21.814858


In [18]:
viewer = napari.view_image(cd31_img, name = 'CD31')
viewer.layers['CD31'].contrast_limits = (0, 500)
mask_layer = viewer.add_labels(cd31_mask, name = 'CD31_mask', blending = 'additive', opacity = 0.5)
tmp_layer = viewer.add_labels(cd31_mask_tmp, name = 'CD31_object', blending = 'additive', opacity = 0.5)

In [57]:
cd31_mask = viewer.layers['CD31_object'].data
#cd31_mask = viewer.layers['CD31_mask'].data
print(cd31_mask.shape)

(2968, 3941)


In [58]:
cd31_mask_path = os.path.join(set_path, img_name.replace('.czi', '_cd31_mask.tif'))
imwrite(cd31_mask_path, cd31_mask)

In [19]:
cd31_mask_tmp = expand_labels(cd31_mask, distance = 15) 
cd31_mask_tmp = np.where(cd31_mask_tmp > 0, 1, 0) 
cd31_mask_tmp = np.uint16(label(cd31_mask_tmp)) 

cd31_df = roi_quant(cd31_mask_tmp, cd31_img) 
cd31_df_path = os.path.join(set_path, img_name.replace('.czi', '_cd31_output.csv')) 
cd31_df.to_csv(cd31_df_path, header = True) 

island_mask_tmp = expand_labels(cd31_mask_tmp, distance = 250) 
island_mask = np.subtract(island_mask_tmp, cd31_mask_tmp) 
island_mask = np.uint16(island_mask) 
island_mask_path = os.path.join(set_path, img_name.replace('.czi', '_island_mask.tif')) 
imwrite(island_mask_path, island_mask)

exp_df = roi_quant(island_mask, exp_img) 
exp_df_path = os.path.join(set_path, img_name.replace('.czi', '_exp_output.csv')) 
exp_df.to_csv(exp_df_path, header = True) 

In [20]:
exp_layer = viewer.add_image(exp_img, name = 'Exp')
island_layer = viewer.add_labels(island_mask, name = 'island_mask', blending = 'additive', opacity = 0.5)

In [61]:
island_mask = viewer.layers['island_mask'].data
island_mask_path = os.path.join(set_path, img_name.replace('.czi', '_island_mask.tif')) 
imwrite(island_mask_path, island_mask)

In [62]:
viewer.close()

### Extra

In [191]:
print(cd31_img.shape)

(19045, 10290)


In [195]:
y_index = round(19045/2)
x_index = round(10290/2)
print(y_index, x_index)

9522 5145


In [196]:
cd31_img_a = cd31_img[0:y_index, 0:x_index]
print(cd31_img_a.shape)

(9522, 5145)


In [197]:
cd31_mask = model_apply(cd31_img_a, cd31_model)

Segmenting
Add original pixel values
Add Gabor features
Add Median filter
Sigma = 2
Sigma = 3
Sigma = 4
Sigma = 5
Sigma = 6
Sigma = 7
Add Entropy filter
Sigma = 2


/Users/tranad/opt/anaconda3/envs/napari-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:262: UserWarning: Bad rank filter performance is expected due to a large number of bins (4091), equivalent to an approximate bitdepth of 12.0.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Sigma = 3


/Users/tranad/opt/anaconda3/envs/napari-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:262: UserWarning: Bad rank filter performance is expected due to a large number of bins (4091), equivalent to an approximate bitdepth of 12.0.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Sigma = 4


...

Sigma = 6


/Users/tranad/opt/anaconda3/envs/napari-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:262: UserWarning: Bad rank filter performance is expected due to a large number of bins (4091), equivalent to an approximate bitdepth of 12.0.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Sigma = 7


/Users/tranad/opt/anaconda3/envs/napari-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:262: UserWarning: Bad rank filter performance is expected due to a large number of bins (4091), equivalent to an approximate bitdepth of 12.0.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Segmentation complete:  2023-04-21 15:23:25.048768


In [198]:
cd31_mask_a = cd31_mask

In [200]:
cd31_img_b = cd31_img[0:y_index, x_index:]
print(cd31_img_b.shape)

(9522, 5145)


In [201]:
cd31_mask = model_apply(cd31_img_b, cd31_model)

Segmenting
Add original pixel values
Add Gabor features
Add Median filter
Sigma = 2
Sigma = 3
Sigma = 4
Sigma = 5
Sigma = 6
Sigma = 7
Add Entropy filter
Sigma = 2


/Users/tranad/opt/anaconda3/envs/napari-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:262: UserWarning: Bad rank filter performance is expected due to a large number of bins (4096), equivalent to an approximate bitdepth of 12.0.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Sigma = 3


/Users/tranad/opt/anaconda3/envs/napari-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:262: UserWarning: Bad rank filter performance is expected due to a large number of bins (4096), equivalent to an approximate bitdepth of 12.0.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Sigma = 4


...

Sigma = 6


/Users/tranad/opt/anaconda3/envs/napari-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:262: UserWarning: Bad rank filter performance is expected due to a large number of bins (4096), equivalent to an approximate bitdepth of 12.0.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Sigma = 7


/Users/tranad/opt/anaconda3/envs/napari-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:262: UserWarning: Bad rank filter performance is expected due to a large number of bins (4096), equivalent to an approximate bitdepth of 12.0.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Segmentation complete:  2023-04-21 15:50:37.045973


In [202]:
cd31_mask_b = cd31_mask

In [203]:
cd31_img_c = cd31_img[y_index:, 0:x_index]
print(cd31_img_c.shape)

(9523, 5145)


In [204]:
cd31_mask = model_apply(cd31_img_c, cd31_model)

Segmenting
Add original pixel values
Add Gabor features
Add Median filter
Sigma = 2
Sigma = 3
Sigma = 4
Sigma = 5
Sigma = 6
Sigma = 7
Add Entropy filter
Sigma = 2


/Users/tranad/opt/anaconda3/envs/napari-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:262: UserWarning: Bad rank filter performance is expected due to a large number of bins (4096), equivalent to an approximate bitdepth of 12.0.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Sigma = 3


/Users/tranad/opt/anaconda3/envs/napari-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:262: UserWarning: Bad rank filter performance is expected due to a large number of bins (4096), equivalent to an approximate bitdepth of 12.0.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Sigma = 4


...

Sigma = 6


/Users/tranad/opt/anaconda3/envs/napari-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:262: UserWarning: Bad rank filter performance is expected due to a large number of bins (4096), equivalent to an approximate bitdepth of 12.0.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Sigma = 7


/Users/tranad/opt/anaconda3/envs/napari-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:262: UserWarning: Bad rank filter performance is expected due to a large number of bins (4096), equivalent to an approximate bitdepth of 12.0.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Segmentation complete:  2023-04-21 16:33:40.594592


In [205]:
cd31_mask_c = cd31_mask

In [206]:
cd31_img_d = cd31_img[y_index:, x_index:]
print(cd31_img_d.shape)

(9523, 5145)


In [207]:
cd31_mask = model_apply(cd31_img_d, cd31_model)

Segmenting
Add original pixel values
Add Gabor features
Add Median filter
Sigma = 2
Sigma = 3
Sigma = 4
Sigma = 5
Sigma = 6
Sigma = 7
Add Entropy filter
Sigma = 2


/Users/tranad/opt/anaconda3/envs/napari-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:262: UserWarning: Bad rank filter performance is expected due to a large number of bins (4045), equivalent to an approximate bitdepth of 12.0.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Sigma = 3


/Users/tranad/opt/anaconda3/envs/napari-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:262: UserWarning: Bad rank filter performance is expected due to a large number of bins (4045), equivalent to an approximate bitdepth of 12.0.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Sigma = 4


...

Sigma = 6


/Users/tranad/opt/anaconda3/envs/napari-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:262: UserWarning: Bad rank filter performance is expected due to a large number of bins (4045), equivalent to an approximate bitdepth of 12.0.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Sigma = 7


/Users/tranad/opt/anaconda3/envs/napari-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:262: UserWarning: Bad rank filter performance is expected due to a large number of bins (4045), equivalent to an approximate bitdepth of 12.0.
  image, footprint, out, mask, n_bins = _preprocess_input(image, footprint,


Segmentation complete:  2023-04-24 09:25:02.243117


In [208]:
cd31_mask_d = cd31_mask

In [210]:
cd31_mask_ab = np.concatenate((cd31_mask_a, cd31_mask_b), axis = 1)
cd31_mask_ab.shape

(9522, 10290)

In [211]:
cd31_mask_cd = np.concatenate((cd31_mask_c, cd31_mask_d), axis = 1)
cd31_mask_cd.shape

(9523, 10290)

In [212]:
cd31_mask = np.concatenate((cd31_mask_ab, cd31_mask_cd), axis = 0)
cd31_mask.shape

(19045, 10290)

In [214]:
cd31_mask = np.uint16(label(cd31_mask))